# Nutrient Recovery and Reuse Model

## M Fein-Cole Honors Thesis
#### Rubenstein School of the Environment and Natural Resources
#### University of Vermont
#### Spring 2021

## Import system modules

In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *

import csv

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

# Turn of geoprocessing log history
arcpy.SetLogHistory(False)

# In geoproccessing options: 
# make sure "Add output datasets to an open map" is unchecked

# Import datetime for monitoring time (IS THIS STILL NEEDED)
from datetime import datetime

## Set environments and import data

In [2]:
# Set environment settings
arcpy.ResetEnvironments()

env.workspace = r"in_memory" # Set workspace to system memory

coord_system = "Asia South Albers Equal Area Conic" # Name of coordinate system here
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(coord_system) # Set coordinate system

# Get paths to input files
recovery_raster = r"C:\Temp\randomrecov3.tif" # Path to recovery raster here
potential_reuse_raster = r"C:\Temp\randomreuse2.tif" # Path to potential reuse raster here

# Geoprocessing environment settings
arcpy.env.overwriteOutput = True
arcpy.env.cellSize = potential_reuse_raster
arcpy.env.snapRaster = potential_reuse_raster
arcpy.env.extent = potential_reuse_raster

# Convert recovery and potential reuse pixel types to signed integer
recovery_raster = arcpy.management.CopyRaster(recovery_raster, r"in_memory\recovery_raster", "", "", "0", "", "", "32_BIT_SIGNED")
potential_reuse_raster = arcpy.management.CopyRaster(potential_reuse_raster, r"in_memory\reuse_raster", "", "", "0", "", "", "32_BIT_SIGNED")
original_potential_reuse = arcpy.management.CopyRaster(potential_reuse_raster, r"in_memory\og_reuse_raster", "", "", "0", "", "", "32_BIT_SIGNED")

#TEMPORARY
recovery_raster = arcpy.management.CopyRaster(recovery_raster, r"C:\Temp\workspace\recovery_raster.tif", "", "", "0", "", "", "32_BIT_SIGNED")
# THIS MIGHT BE TEMPORARY
reuse_sources = CreateConstantRaster(0, "INTEGER")

# Build an attribute table for recovery raster
recovery_table = arcpy.BuildRasterAttributeTable_management(recovery_raster)

# Identify maximum and minimum values in recovery layer
in_layer = recovery_table
fieldName = "VALUE"
recovery_table_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
greatest_recovery = max(recovery_table_list) # maximum recovery amount
smallest_recovery = min(recovery_table_list) # minimum recovery amount
recovery = float(smallest_recovery)
recovery = int(recovery) # value of smallest recovery location

# Set iterator variables for later use
go_on = True # are there more recovery locations with resources to be distributed? 
continue_at_site = False # is there remaining value at the current loccation?

iteration = 0 # how many loops the model has done
site_num = 0 # how many recovery locations has the model gone through

## Model loop

In [3]:
# Get and print time of start
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start: ", current_time)

while go_on: 

    if continue_at_site: # Don't need to re-select smallest site
        iteration = iteration + 1

    else: # Need to select new smallest recovery location
        iteration = iteration + 1
        site_num = site_num + 1

    ### IDENTIFY SMALLEST RECOVERY LOCATION

    # Use select by attributes to identify the next smallest recovery location (target recovery location)
    in_layer = recovery_table
    fieldName = "Value"
    recovery_table_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
    sqlExp = fieldName + '=' + str(min(recovery_table_list))
    recovery = int(min(recovery_table_list))
    input_layer = recovery_raster
    recovery_selection = ExtractByAttributes(input_layer, sqlExp)

    # Zonal statistic of recovery layer of recovery selection to capture if there are locations with the same recovery amount
    in_zone_data = recovery_selection
    zone_field = "Value"
    in_value_raster = recovery_raster
    out_table2 = r"in_memory\recov_sum_table"
    ignore_nodata = "DATA"
    statistics_type = "SUM"
    recov_sum_table = ZonalStatisticsAsTable(in_zone_data, zone_field, in_value_raster, out_table2, ignore_nodata, statistics_type)

    # Create an integer variable for recovery, accounting for multiple sites with the same recovery value
    in_layer = r"in_memory\recov_sum_table"
    fieldName = "SUM"
    recov_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
    total_recov = max(recov_list)
    total_recov = int(total_recov)

    # Create a uniform raster with value equal to the value at the target recovery location
    constant_ras = CreateConstantRaster(total_recov, "INTEGER")

    ### IDENTIFY POTENTIAL REUSE LOCATIONS CLOSEST TO THE SELECTED RECOVERY LOCATION

    # Use Euclidean distance to determine distances between target recovery locations and potential reuse locations
    in_source_data = recovery_selection
    maximum_distance = ""
    cell_size = potential_reuse_raster
    outEucDist = EucDistance(in_source_data, maximum_distance, cell_size) # Distance from smallest recovery location

    # Set any values in outEucDist null if there is no potential reuse at those locations
    reuse_null = SetNull(potential_reuse_raster, 1, "Value=0") # If there is potential reuse, value = 1, else it is null 
    zeros_eucdist = outEucDist * reuse_null
    outCon = Con(IsNull(zeros_eucdist), -5, zeros_eucdist) # Set those nulls equal to -5
    outCon2 = arcpy.management.CopyRaster(outCon, r"in_memory\outCon2.tif", "", "", "", "", "", "32_BIT_SIGNED") # convert pixel type to signed integer
    int_zeros_euc = SetNull(outCon2, outCon2, "Value = -5") # Convert the nulls back to -5. 
    int_zeros_euc = SetNull(int_zeros_euc, int_zeros_euc, "Value = -5") # Convert the nulls back to -5. 
    # zeros_eucdist has been converted to integer pixel type, zeros have remained zeros and nulls remain null
    euc_table = arcpy.BuildRasterAttributeTable_management(int_zeros_euc) # Build attribute table for int_zeros_euc

    # Use select by attributes to identify potential reuse pixels locations to the recovery point  (target reuse locations)
    in_layer = euc_table
    fieldName = "Value"
    dist_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
    sqlExp = fieldName + '=' + str(min(dist_list))
    input_layer = int_zeros_euc
    min_dist_select = ExtractByAttributes(input_layer, sqlExp) 
    # Convert min_dist_select pixel type to signed integer
    min_dist_select1 = arcpy.management.CopyRaster(min_dist_select, r"in_memory\min_dist_sel.tif", "", "", "", "", "", "32_BIT_SIGNED")
    min_dist = min(dist_list)
    
    # Use zonal statistics (sum function) to get a single value for potential reuse at all target reuse locations
    in_zone_data = min_dist_select1
    zone_field = "VALUE"
    in_value_raster = potential_reuse_raster
    statistics_type = "SUM"
    ignore_nodata = "DATA"
    outzstat = ZonalStatistics(in_zone_data, zone_field, in_value_raster, statistics_type, ignore_nodata)

    # Generate same zonal statistics as above in table form to create a variable for summed target reuse locations
    in_zone_data = min_dist_select1
    zone_field = "VALUE"
    in_value_raster = potential_reuse_raster
    out_table2 = r"in_memory\reusesumtable"
    ignore_nodata = "DATA"
    statistics_type = "SUM"
    reusesumtable = ZonalStatisticsAsTable(in_zone_data, zone_field, in_value_raster, out_table2, ignore_nodata, statistics_type)

    # Create an integer variable for total potential reuse at target reuse locations
    in_layer = r"in_memory\reusesumtable"
    fieldName = "SUM"
    reuse_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
    current_site_reuse = max(reuse_list)
    reuse = float(current_site_reuse)
    reuse = round(reuse)
    reuse = int(reuse)
    # Create a constant raster of nearby reuse
    constant_ras2 = CreateConstantRaster(reuse, "INTEGER")

    ### "DISTRIBUTE" RECOVERED MATERIAL FROM TARGET RECOVERY LOCATION TO TARGET REUSE LOCATIONS

    # Create a binary raster where target locations have values of 0 and non-target locations have values of 1
    binary_rast = Con(IsNull(min_dist_select1), 1, 0)
    binary_rast_recov = Con(IsNull(recovery_selection), 1, 0)

    if reuse >= total_recov: # Less material recovered than could be reused at target locations

        # Update recovery raster
        recovery_raster = Raster(recovery_raster) * binary_rast_recov # Value at target recovery location is changed to zero

        # Update potential reuse raster
        reuse_percent = Raster(potential_reuse_raster) / outzstat # Proportion of potential reuse at each location compared to total
        cell_distr = reuse_percent * Raster(constant_ras) # Proportional amount of material available for each target recovery location
        cell_distr2 = Int(cell_distr + 0.5) # Round to nearest integer
        update_reuse = Raster(potential_reuse_raster) - cell_distr2 # Remaining amount of potential recovery after distribution
        update_reuse2 = Con(IsNull(update_reuse), 0, update_reuse) # Set nulls to zero
        almost_potential_reuse_raster = Raster(potential_reuse_raster) *  binary_rast # Value at target reuse locations are changed to zero
        potential_reuse_raster = almost_potential_reuse_raster + update_reuse2 # Replace target reuse cells with new values

        continue_at_site = False # No more value at current target recovery location

        distributed = total_recov
        
    if reuse < total_recov: 

        # Update recovery raster
        if total_recov == recovery:
            new_recovery = recovery_selection - constant_ras2 # Subtract distributed material from target recovery location
            new_recovery2 = Con(IsNull(new_recovery), 0, new_recovery) # Set nulls to zero
            almost_recovery_raster = Raster(recovery_raster) *  binary_rast_recov # Value at target recovery locations is changed to zero
            recovery_raster = almost_recovery_raster + new_recovery2 # Replace target recovery cells with new value

        else:
            new_recovery = recovery_selection - potential_reuse_raster # Subtract distributed material from target recovery location
            new_recovery2 = Con(IsNull(new_recovery), 0, new_recovery) # Set nulls to zero
            almost_recovery_raster = Raster(recovery_raster) *  binary_rast_recov # Value at target recovery locations is changed to zero
            recovery_raster = almost_recovery_raster + new_recovery2 # Replace target recovery cells with new value

        # Update potential reuse raster
        potential_reuse_raster = Raster(potential_reuse_raster) * binary_rast

        continue_at_site = True # More value at current target recovery location
        
        distributed = reuse

    # Calculate and save distribution raster
    outCon3 = Con(IsNull(potential_reuse_raster), 0, potential_reuse_raster)
    difference_raster = original_potential_reuse - outCon3
    difference_raster = SetNull(difference_raster, difference_raster, "Value = 0")

    # Create a spreadsheet containing distance travelled
    fields = [iteration, min_dist, distributed]
    with open(r'Z:\nutrient_model\distance_travelled6.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)
    
    ### PREPARE FOR NEXT ITERATION
    #CONFIRM THAT THERE ARE STILL RECOVERY LOCATIONS THAT NEED TO BE DISTRIBUTED AND THAT THE MODEL SHOULD CONTINUE
    
    try: 
        # Convert recovery raster pixel type to signed integer 
        recovery_raster = arcpy.management.CopyRaster(recovery_raster, r"in_memory\recovery_raster.tif", "", "", "0", "", "", "32_BIT_SIGNED") 

        # Set zeros in input raster null
        in_raster = recovery_raster
        in_constant = recovery_raster
        where_clause = "VALUE = 0"

        recovery_raster = SetNull(in_raster, in_constant, where_clause)

        # Create attribute table for recovery raster
        recovery_table = arcpy.BuildRasterAttributeTable_management(recovery_raster)

        # Confirm that value remains in the recovery raster

        in_layer = recovery_table
        fieldName = "VALUE"
        rem_recovery_table_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
        max_remaining_recov = max(rem_recovery_table_list)
        smallest_recovery = min(rem_recovery_table_list)
        recovery = float(smallest_recovery)
        recovery = int(recovery)

        go_on = True

    except: # Set Null will fail if all recovered resource has been "distributed"
        go_on = False
        break # Stop distribution loop
    
    ### SAVE INTERMEDIATES TO MEMORY TO IMPROVE PROCESSING TIMES

    # Save intermediates to memory
    recov_path_name = 'in_memory\_recovery_rast' + str(iteration) +'.tif'
    reuse_path_name = 'in_memory\_reuse_rast' + str(iteration) +'.tif'
    recovery_raster.save(recov_path_name)
    potential_reuse_raster.save(reuse_path_name)

    # Reassign variables to saved intermediates
    recovery_raster = recov_path_name # Path to recovery raster here
    potential_reuse_raster = reuse_path_name # Path to potential reuse raster here

    # Convert recovery raster pixel type to signed integer 
    int_recov_path_name = 'in_memory\int_recovery_rast' + str(iteration) + '.tif'
    recovery_raster = arcpy.management.CopyRaster(recovery_raster, int_recov_path_name, "", "", "0", "", "", "32_BIT_SIGNED")
    # Convert potential reuse raster pixel type to signed integer 
    int_reuse_path_name = 'in_memory\int_reuse_rast' + str(iteration) + '.tif'
    potential_reuse_raster = arcpy.management.CopyRaster(potential_reuse_raster, int_reuse_path_name, "", "", "0", "", "", "32_BIT_SIGNED")

    recovery_table = arcpy.BuildRasterAttributeTable_management(recovery_raster)

    in_layer = recovery_table
    fieldName = "VALUE"
    recovery_table_list = [r[0] for r in arcpy.da.SearchCursor (in_layer, [fieldName])]
    max_remaining_recov = max(recovery_table_list)
    smallest_recovery = min(recovery_table_list)
    recovery = float(smallest_recovery)
    recovery = int(recovery)
        
### SAVE THE CHANGE IN POTENTIAL REUSE AFTER DISTRIBUTION HAS STOPPED
outCon3 = Con(IsNull(potential_reuse_raster), 0, potential_reuse_raster)
difference_raster = original_potential_reuse - outCon3
difference_raster = SetNull(difference_raster, difference_raster, "Value = 0")
difference_raster.save(r"C:\Temp\difference_raster6.tif")

reuse_sources.save(r"C:\Temp\workspace\reuse_sources.tif")

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End: ", current_time)

Start:  10:54:11
End:  11:27:37
